#### informacoes dos dados
<p> inicio de medicao do adcp: 2016-11-18 20:00:00</p>
<p> fim de medicao adcp : 2017-12-04 13:00:00</p>

### inicio da analise

In [32]:
import datetime as dt
import matplotlib.pyplot as plt
import pandas as pd, numpy as np
import matplotlib.dates as mdates

In [56]:
from filters import moving_avg

In [81]:
%matplotlib auto

Using matplotlib backend: Qt5Agg


<h3>Funcoes</h3>

In [34]:
def corta_serie_temporal(date_min, date_max, vetor_tempo, data):

    date_min = dt.datetime.strptime(date_min,
                                    "%Y-%m-%d %H:%M:%S")
    
    date_max = dt.datetime.strptime(date_max,
                                    '%Y-%m-%d %H:%M:%S')
    vetor_tempo = vetor_tempo
    
    rc = data.copy()
    
    condition = ((vetor_tempo >= date_min) & (vetor_tempo <= date_max)).squeeze()
    

    rc = rc.loc[condition, :]
    return rc

<h3> carregamento dos dados </h3>

In [35]:
path = '/home/gus/Documentos/mestrado/dados-mestrado/vento-inmet'
dfwind = pd.read_csv(path+'/dados_rotacionados_dir_oceanografica.csv', sep=";")

In [36]:
#-- vetor com os tempos dos dados de vento --
vetor_time = [dt.datetime.strptime(ii,'%Y-%m-%d %H:%M:%S')
              for ii in dfwind.time]

vetor_time = pd.DataFrame(vetor_time)

In [37]:
path = '/home/gus/Documentos/mestrado/dados-mestrado/dados-adcp-completos'

# --
adcpu = pd.read_csv(path+'/perpendicular-2016-2017.csv', delimiter= ';')
adcpv = pd.read_csv(path+'/paralelo-2016-2017.csv', delimiter= ';')
# --

In [38]:
# -- checando as datas limites dos dados da boia --

adcpu.time.iloc[0], adcpu.time.iloc[-1]

('2016-11-18 20:00:00', '2017-12-04 13:00:00')

In [39]:
dfwind.time.iloc[0], dfwind.time.iloc[-1]

('2016-12-01 00:00:00', '2017-12-31 23:00:00')

#### Ajuste das series temporais

In [40]:
# - vento
rc = corta_serie_temporal('2016-11-18 20:00:00','2017-12-04 13:00:00',
                         vetor_time, dfwind)

In [41]:
# - tempo do adcp
adcp_time = adcpu.time.iloc[:]

In [42]:
# - tempo do adcp para datetime
adcp_timedt = pd.DataFrame([dt.datetime.strptime(
    ii,'%Y-%m-%d %H:%M:%S') for ii in adcp_time])

In [43]:
# - adcp
adcpu = corta_serie_temporal(dfwind.time.iloc[0],
                             dfwind.time.iloc[-1], adcp_timedt , adcpu)

adcpv = corta_serie_temporal(dfwind.time.iloc[0],
                             dfwind.time.iloc[-1], adcp_timedt , adcpv)


In [44]:
dfwind = rc;
del rc

In [45]:
if len(dfwind) == len(adcpu):
    print("As series estão ajustadas")

As series estão ajustadas


### correlação

In [46]:
dfwind.head()

,wu,wv,wgu,wgv,time
0,-1.626940,0.056814,-4.367049,0.152501,2016-12-01 00:00:00
1,-2.136798,-0.149419,-4.957370,-0.346653,2016-12-01 01:00:00
2,-2.055082,0.071765,-5.308962,0.185393,2016-12-01 02:00:00
3,-2.053517,-0.107620,-5.989423,-0.313892,2016-12-01 03:00:00
4,-2.994712,0.156946,-6.417239,0.336313,2016-12-01 04:00:00


## rotacionando os dados de vento

In [47]:
wul = dfwind.wu*np.cos(np.deg2rad(60)) + dfwind.wv*np.sin(np.deg2rad(60))
wvl = -dfwind.wu*np.sin(np.deg2rad(60)) + dfwind.wv*np.cos(np.deg2rad(60))

dfwind['wul'] = wul
dfwind['wvl'] = wvl

del wul, wvl

### checando os dataframes

In [48]:
dfwind.head()

,wu,wv,wgu,wgv,time,wul,wvl
0,-1.626940,0.056814,-4.367049,0.152501,2016-12-01 00:00:00,-0.764268,1.437378
1,-2.136798,-0.149419,-4.957370,-0.346653,2016-12-01 01:00:00,-1.197800,1.775811
2,-2.055082,0.071765,-5.308962,0.185393,2016-12-01 02:00:00,-0.965391,1.815636
3,-2.053517,-0.107620,-5.989423,-0.313892,2016-12-01 03:00:00,-1.119960,1.724587
4,-2.994712,0.156946,-6.417239,0.336313,2016-12-01 04:00:00,-1.361436,2.671970


In [49]:
adcpu.head(3)

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,time
292,2.766456,3.854003,3.311954,3.635609,6.132167,3.430062,1.421306,4.360670,3.294062,2.445067,...,4.545147,5.108417,2.468644,-2.675337,-9.637806,-17.170062,NaN,-35.000894,-74.319552,2016-12-01 00:00:00
293,-3.342730,-1.956765,-1.315994,-0.333824,-2.800309,-1.315994,-2.278983,-1.256592,-2.117925,-0.680900,...,-1.032690,-1.426545,-1.699171,-0.817927,-4.404494,-5.231856,-12.336748,-31.245520,-49.148609,2016-12-01 01:00:00
294,-4.346033,-3.469523,-5.694168,-6.424747,-5.896873,-2.076696,-2.593048,-1.471355,-4.295756,-5.786307,...,-3.527156,-2.480307,-4.511584,-6.892748,-8.873202,-14.437034,-20.847654,-30.398226,-49.930014,2016-12-01 02:00:00


In [99]:
#x = moving_avg(dfwind.wul, 6)
#y = moving_avg(dfwind.wvl, 6)

#dfwind['wul12'] = x
#dfwind['wvl12'] = y
#del x, y

## A 1ª coluna será destinada a comp. perpendicular das correntes,
## enqunto a 2ª coluna será destinada a comp. paralela das correntes.

In [95]:

correlacao_wu = np.zeros([20, 2])

correlacao_wu.fill(np.NaN)

correlacao_wv = correlacao_wu.copy()

In [96]:
for iterador in range(20):
    
    # vento perpendicular 
    R1 = dfwind['wul12'].corr(adcpu.iloc[:, iterador]
                              , method = 'pearson')
    
    R2 = dfwind['wul12'].corr(adcpv.iloc[:, iterador]
                              , method = 'pearson')
    
    # vento paralelo
    R3 = dfwind['wvl12'].corr(adcpu.iloc[:, iterador]
                              , method = 'pearson')
    R4 = dfwind['wvl12'].corr(adcpv.iloc[:, iterador]
                              , method = 'pearson')
    
    
    correlacao_wu[iterador, 0] = R1
    correlacao_wu[iterador, 1] = R2
    
    correlacao_wv[iterador, 0] = R3
    correlacao_wv[iterador, 1] = R4
    
    del R1, R2, R3, R4

In [97]:
correlacao_wu

array([[ 0.01298291,  0.20293965],
       [ 0.00703502,  0.23502264],
       [ 0.04115776,  0.24014538],
       [ 0.06516627,  0.24378786],
       [ 0.06734328,  0.24540279],
       [ 0.07040077,  0.25126114],
       [ 0.06375428,  0.25608827],
       [ 0.05904064,  0.25651971],
       [ 0.05430566,  0.25915675],
       [ 0.03829569,  0.26166713],
       [ 0.01696988,  0.26797328],
       [ 0.00492435,  0.26740358],
       [ 0.00045378,  0.26956061],
       [-0.0234494 ,  0.24502032],
       [-0.01675731,  0.21861395],
       [ 0.00826836,  0.1869312 ],
       [ 0.04374824,  0.17000087],
       [ 0.07409263,  0.17325939],
       [ 0.11662446,  0.1627719 ],
       [ 0.25547091,  0.2211642 ]])

In [98]:
correlacao_wv

array([[ 0.03605139, -0.16999298],
       [-0.01496868, -0.19767374],
       [-0.03530759, -0.21171301],
       [-0.05275257, -0.22368166],
       [-0.06241408, -0.2263774 ],
       [-0.06245528, -0.23379127],
       [-0.06940288, -0.23701833],
       [-0.07283641, -0.22991971],
       [-0.0744422 , -0.22996157],
       [-0.0727191 , -0.22681474],
       [-0.05887759, -0.21380317],
       [-0.04463119, -0.20089461],
       [-0.04091469, -0.17322733],
       [-0.04350752, -0.12833953],
       [-0.04591774, -0.08657572],
       [-0.05850667, -0.03956401],
       [-0.06536394, -0.0243812 ],
       [-0.07367734, -0.01882027],
       [-0.07635463, -0.01050023],
       [-0.06648077,  0.00378545]])